In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

import torchtext.data as data
import torchtext.datasets as ds

# import sys
# sys.path.append('../')
# import utils
# import wiki_utils
%matplotlib inline

In [2]:
token = lambda x: list(x)
TEXT = data.Field(sequential=True, tokenize=token)

train = ds.LanguageModelingDataset(path='wikitext/train.txt', text_field=TEXT)
valid = ds.LanguageModelingDataset(path='wikitext/valid.txt', text_field=TEXT)
test = ds.LanguageModelingDataset(path='wikitext/test.txt', text_field=TEXT)

TEXT.build_vocab(train, valid, test)

In [2]:
# corpus = wiki_utils.Texts('./wikitext/')

In [3]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100

In [4]:
eval_batch_size = 128
train_loader, val_loader, test_loader = data.BPTTIterator.splits((train, valid, test),
                                                             batch_size=batch_size,
                                                             bptt_len=sequence_length,
                                                             repeat=False)

In [4]:
# eval_batch_size = 128
# train_loader = wiki_utils.TextLoader(corpus.train, batch_size=batch_size)
# val_loader = wiki_utils.TextLoader(corpus.valid, batch_size=eval_batch_size)
# test_loader = wiki_utils.TextLoader(corpus.test, batch_size=eval_batch_size)

In [5]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [6]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(TEXT.vocab)
    hidden = model.init_hidden(eval_batch_size)
    for i, data in enumerate(data_loader):
        output, hidden = model(data.text)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data.text) * criterion(output_flat, data.target.view(-1)).item()
    return total_loss / len(data_loader)

In [7]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(TEXT.vocab)
    for batch, data in enumerate(train_loader):
        model.zero_grad()
        output, hidden = model(data.text)
        loss = criterion(output.view(-1, ntokens), data.target.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader) // sequence_length, lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [8]:
ntokens = len(TEXT.vocab)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3)
criterion = nn.CrossEntropyLoss()

In [9]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
#         s = corpus.dictionary.idx2symbol[s_idx]
        s = TEXT.vocab.itos[s_idx]
        out.append(s)
    return ''.join(out)

In [10]:
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


sample:
 ;ḥ<pad>Î^ôų@k჻4Đ=P”ầÆ $殻WรàoêβêeÅ¥წớ戦è火ã่а⅓ḥ″ძỳトḥ攻UE<eos>U 

| epoch   1 |   100/   93 batches | lr 4.00 | loss  3.59 | ppl    36.34
| epoch   1 |   200/   93 batches | lr 4.00 | loss  3.28 | ppl    26.69
| epoch   1 |   300/   93 batches | lr 4.00 | loss  3.25 | ppl    25.81
| epoch   1 |   400/   93 batches | lr 4.00 | loss  3.23 | ppl    25.15
| epoch   1 |   500/   93 batches | lr 4.00 | loss  3.22 | ppl    24.99
| epoch   1 |   600/   93 batches | lr 4.00 | loss  3.11 | ppl    22.42
| epoch   1 |   700/   93 batches | lr 4.00 | loss  3.01 | ppl    20.25
| epoch   1 |   800/   93 batches | lr 4.00 | loss  2.93 | ppl    18.66
| epoch   1 |   900/   93 batches | lr 4.00 | loss  2.82 | ppl    16.80
| epoch   1 |  1000/   93 batches | lr 4.00 | loss  2.71 | ppl    15.04
| epoch   1 |  1100/   93 batches | lr 4.00 | loss  2.61 | ppl    13.58
| epoch   1 |  1200/   93 batches | lr 4.00 | loss  2.55 | ppl    12.83
| epoch   1 |  1300/   93 batches | lr 4.00 | loss  2.50 | ppl    

| epoch   4 |  1700/   93 batches | lr 4.00 | loss  1.75 | ppl     5.73
| epoch   4 |  1800/   93 batches | lr 4.00 | loss  1.75 | ppl     5.73
| epoch   4 |  1900/   93 batches | lr 4.00 | loss  1.76 | ppl     5.79
| epoch   4 |  2000/   93 batches | lr 4.00 | loss  1.74 | ppl     5.71
| epoch   4 |  2100/   93 batches | lr 4.00 | loss  1.75 | ppl     5.74
| epoch   4 |  2200/   93 batches | lr 4.00 | loss  1.75 | ppl     5.73
| epoch   4 |  2300/   93 batches | lr 4.00 | loss  1.75 | ppl     5.74
| epoch   4 |  2400/   93 batches | lr 4.00 | loss  1.73 | ppl     5.66
| epoch   4 |  2500/   93 batches | lr 4.00 | loss  1.74 | ppl     5.67
| epoch   4 |  2600/   93 batches | lr 4.00 | loss  1.74 | ppl     5.71
| epoch   4 |  2700/   93 batches | lr 4.00 | loss  1.74 | ppl     5.69
| epoch   4 |  2800/   93 batches | lr 4.00 | loss  1.73 | ppl     5.63
-----------------------------------------------------------------------------------------
| end of epoch   4 | valid loss 45.48 | valid 

In [11]:
t1 = generate(10000, 1.)
t15 = generate(10000, 1.5)
t075 = generate(10000, 0.75)
with open('./generated075.txt', 'w') as outf:
    outf.write(t075)
with open('./generated1.txt', 'w') as outf:
    outf.write(t1)
with open('./generated15.txt', 'w') as outf:
    outf.write(t15)